In [ ]:
1R-1C model


3R-1C model

5R-1C model (ISO7751)

carpet plot: heating vs cooling


carpet plot: occupancy vs building age


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import cdecimal as dec
import re as re
import csv
import os, sys
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
import plotly.graph_objs as go


In [3]:
paths = {}
def filepath(filename):
    paths['main'] = "C:\Users\Zghiru\Documents\GitHub\ASF_Simulation\Simulation_Tool\New_SimulationEnvironment\CEA_Archetypes_CH"
    #os.path.abspath(os.path.dirname(sys.argv[0]))
    paths['result'] = os.path.join(paths['main'], filename)  
    return paths['result']

def split_Name_char_num(dataframe):
    occtype=[]
    typenum=[]
    for name in dataframe.Name: 
        match = re.match(r"([a-zA-Z_]+)([0-9]+)", name, re.I)
        if match:
            items = match.groups()
            occtype.append(items[0])
            typenum.append(int(items[1]))
    dataframe['occtype'] = pd.DataFrame(occtype)
    dataframe['typenum'] = pd.DataFrame(typenum)
    dataframe.sort_values(['occtype','typenum'],ascending=[False,True])
    return 


In [15]:
b_props = pd.read_csv(filepath('b_props.csv'))
#b_props = b_props[b_props.interval == "2005-2020"] #select buildings from specific period
#b_props = b_props[b_props.type=='construction' #select only new built buildings



cairo_south = pd.read_csv(filepath('Archetypes_Cairo_South.csv'))
cairo_south = split_Name_char_num(cairo_south)
cairo_south = cairo_south.loc[cairo_south['Name'].isin(b_props.Name)]
cairo_south.sort_values(['occtype','typenum'],ascending=[False,True])
#cairo_south['index'] = cairo_south.index # create a sort index for 
#cairo_south.sort_values(['E_HCL'],ascending=True)

cairo_west = pd.read_csv(filepath('Archetypes_Cairo_West.csv'))
cairo_west = split_Name_char_num(cairo_west)
cairo_west = cairo_west.loc[cairo_west['Name'].isin(b_props.Name)]
cairo_west.sort_values(['occtype','typenum'],ascending=[False,True])

cw = cairo_west.drop(['Name', 'occtype', 'typenum'],axis=1)
cs = cairo_south.drop(['Name', 'occtype', 'typenum'],axis=1)

cairo_WS = cs.subtract(cw)
cairo_WS["Name"]=cairo_south["Name"]

zurich_south = pd.read_csv(filepath('Archetypes_Zurich_South_2013.csv'))
zurich_south = split_Name_char_num(zurich_south)
zurich_south = zurich_south.loc[zurich_south['Name'].isin(b_props.Name)]
zurich_south.sort_values(['occtype','typenum'],ascending=[False,True])

print cairo_south

AttributeError: 'NoneType' object has no attribute 'loc'

In [67]:
# E_HCL across the different orientations
### SUBTRACT PV
zurich = go.Scatter(
    x=zurich_south.Name,
    y=zurich_south.E_HCL,
    fill='tonexty',
    name='Zurich_South'
)
cairosouth = go.Scatter(
    x=cairo_south.Name,
    y=cairo_south.E_HCL,
    fill='tonexty',
    name='Cairo_South'
)
cairowest = go.Scatter(
    x=cairo_west.Name,
    y=cairo_west.E_HCL,
    fill='tonexty',
    name='Cairo_west'
)

data = [zurich,cairosouth,cairowest]

layout = go.Layout(
    title='E_HCL',
    xaxis=dict(
        title='Archetype',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='MWH/year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='new')

In [66]:
#Cairo West-south comparison

cws1 = go.Scatter(
    x=cairo_WS.Name,
    y=cairo_WS.L,
    fill='tonexty',
    name='Lighting'
)
cws2 = go.Scatter(
    x=cairo_WS.Name,
    y=cairo_WS.H,
    fill='tonexty',
    name='Heating'
)
cws3 = go.Scatter(
    x=cairo_WS.Name,
    y=cairo_WS.C,
    fill='tonexty',
    name='Cooling'
)
cws4 = go.Scatter(
    x=cairo_WS.Name,
    y=cairo_WS.E,
    fill='tonexty',
    name='Electricity'
)
cws5 = go.Scatter(
    x=cairo_WS.Name,
    y=cairo_WS.E_HCL,
    fill='tonexty',
    name='Total Energy'
)

cws6 = go.Scatter(
    x=cairo_WS.Name,
    y=cairo_WS.PV,
    fill='tonexty',
    name='PV Generation'
)


data = [cws1,cws2,cws3,cws4,cws5,cws6]


py.iplot(data, filename='horizontal-bar')
layout = go.Layout(
    title='Difference between West and South-oriented rooms in Cairo',
    xaxis=dict(
        title='Archetype',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='MWH/year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='new')

In [63]:
#horizontal stacked barchart for Zurich South

import plotly.plotly as py
import plotly.graph_objs as go

zsl = go.Bar(
    x=zurich_south.L,
    y=zurich_south.Name,
    name='Lighting',
    orientation = 'h'
)
zsh = go.Bar(
    x=zurich_south.H,
    y=zurich_south.Name,
    name='Heating',
    orientation = 'h'
)
zsc = go.Bar(
    x=zurich_south.C,
    y=zurich_south.Name,
    name='Cooling',
    orientation = 'h'
)
zse = go.Bar(
    x=zurich_south.E,
    y=zurich_south.Name,
    name='Electricity',
    orientation = 'h'
)

data = [zsh,zsc,zsl,zse]
layout = go.Layout(
    barmode='stack',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Zurich South')


In [64]:
import plotly.plotly as py
import plotly.graph_objs as go

zsl = go.Bar(
    x=zurich_south.Name,
    y=zurich_south.L,
    name='Lighting',
)
zsh = go.Bar(
    x=zurich_south.Name,
    y=zurich_south.H,
    name='Heating',
)
zsc = go.Bar(
    x=zurich_south.Name,
    y=zurich_south.C,
    name='Cooling',
)
zse = go.Bar(
    x=zurich_south.Name,
    y=zurich_south.E,
    name='Electricity',
)

data = [zsh,zsc,zsl,zse]
layout = go.Layout(
    barmode='stack',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Zurich South_vertical')


In [68]:
import numpy as np

t = np.linspace(-1, 1.2, 2000)
x = cairo_south.E_HCL
y = cairo_west.E_HCL


colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

fig = FF.create_2D_density(
    x, y, colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=3
)

py.iplot(fig, filename='CAIRO West vs South')